In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Import SPY data
spy = pd.read_csv("../input/sp500-index/sp500.csv",index_col="Date")
spy.head()
#spy.shape
#spy.tail()

In [ ]:
dataset_new = ftse.merge(spy,on='Date',how='left',left_index=True,right_index=False, indicator=True)
#dataset_new.shape
dataset_new.head()
dataset_new.tail()

In [ ]:
dataset_new.corr()

In [ ]:
spy['Adj Close'].min()

In [ ]:
spy['Adj Close'].mean()

In [ ]:
spy['Adj Close'].max()

In [ ]:
spy['Adj Close'].std()

In [ ]:
df = pd.read_csv("../input/sp500-index/sp500.csv",index_col='Date')
df.tail()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=2589, color='g', linestyle='-')
plt.axhline(y=2382, color='r', linestyle='-')
plt.axhline(y=2873, color='r', linestyle='-')

In [ ]:
df.mean()

In [ ]:
df.min()

In [ ]:
df.max()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=2180, color='g', linestyle='-')
plt.axhline(y=1575, color='r', linestyle='-')
plt.axhline(y=2867, color='r', linestyle='-')

In [ ]:
spy.head()

In [ ]:
df.corr()

In [ ]:
dataset = df.drop(['Close'],axis=1)
dataset.tail()

In [ ]:
dataset.shape

In [ ]:

from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
#LinearRegression, BayesianRidgeRegression
# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
#X_filled_softimpute = SoftImpute().complete(df)
# using MICE
#df1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='random',n_pmm_neighbors=5).complete(dataset) #probablistic moment matching
df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='random',n_pmm_neighbors=5).complete(dataset) #fill in with samples from posterior predictive distribution

In [ ]:
df2 = pd.DataFrame(df1,columns=('Open','High','Low','Adj Close','Volume'))

In [ ]:
df2.corr()

In [ ]:
df2.drop(['Volume'],axis=1,inplace=True)

In [ ]:
df2.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=2180, color='g', linestyle='-')
plt.axhline(y=1575, color='r', linestyle='-')
plt.axhline(y=2867, color='r', linestyle='-')

In [ ]:
dataset_train = df2.iloc[0:910,]
dataset_train.head()
dataset_train.shape

In [ ]:
dataset_test = df2.iloc[910:1297,]
dataset_test.head()
dataset_test.shape

In [ ]:
training_set = dataset_train.values

In [ ]:
# perform feature scaling 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 910):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 4)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# Making predictions and visualising the results
# Getting the real SPY prices for the period 28/03/2018 to 11/05/2018
real_spy_price = dataset_test['Open'].values

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Open','High','Low','Adj Close']], dataset_test[['Open','High','Low','Adj Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 446):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_spy_price = regressor.predict(X_test)

In [ ]:
#inputs.shape
df_zeros = np.zeros(shape=(len(predicted_spy_price), 4) )
df_zeros[:,0] = predicted_spy_price[:,0]

In [ ]:
predicted_spy_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
# Visualising the results
plt.plot(real_spy_price, color = 'red', label = 'Real SPY Price')
plt.plot(predicted_spy_price, color = 'blue', label = 'Predicted SPY Price')
plt.title('SPY Price Prediction')
plt.xlabel('Time')
plt.ylabel('SPY Price')
plt.legend()
plt.show()

In [ ]:
# import the dataset
#dataset = pd.read_csv("../input/ftse-100-index2/ftse2.csv")
#dataset.head()
#dataset.tail()

In [ ]:
dataset['Adj Close'].mean()

In [ ]:
dataset['Adj Close'].max()

In [ ]:
dataset['Adj Close'].min()

In [ ]:
dataset['Adj Close'].mean()

In [ ]:
dataset['Adj Close'].max()

In [ ]:
dataset['Adj Close'].std()

In [ ]:
df = pd.read_csv("../input/ftse100/ftse.csv",index_col='Date')
df.head()

In [ ]:
#df.tail(40)

In [ ]:
df2 = df.fillna(method='ffill')

In [ ]:
df2.tail()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=7402, color='g', linestyle='-')
plt.axhline(y=7788, color='r', linestyle='-')
plt.axhline(y=6888, color='r', linestyle='-')

In [ ]:
df2.shape

In [ ]:
df.corr()

In [ ]:
# imputation using the mean values
from sklearn.preprocessing import Imputer
imputer = Imputer()
df1=imputer.fit_transform(df)

In [ ]:
# Other Imputation methods
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
#LinearRegression, BayesianRidgeRegression
# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
#X_filled_softimpute = SoftImpute().complete(df)
# using MICE
#df1 = MICE(n_imputations=1000,impute_type='pmm',init_fill_method='random',n_pmm_neighbors=5).complete(df) #probablistic moment matching
df1 = MICE(n_imputations=1000,impute_type='col',init_fill_method='random',n_pmm_neighbors=5).complete(df) #fill in with samples from posterior predictive distribution

In [ ]:
df2 = pd.DataFrame(df1,columns=('Open','High','Low','Close','Adj Close','Volume'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=7402, color='g', linestyle='-')
plt.axhline(y=7788, color='r', linestyle='-')
plt.axhline(y=6888, color='r', linestyle='-')

In [ ]:
df2.tail()

In [ ]:
df2.drop(['Close','Volume'],axis=1,inplace=True)

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
# append SPY Open in FTSE
spy_open = spy.iloc[:,0:2]
spy_open = spy_open.rename(columns={'Open':'Open_spy'})
spy_open.head()


In [ ]:
dataset_new = dataset.merge(spy_open,on='Date',how='inner',left_index=True,right_index=False, indicator=True)
dataset_new.shape

In [ ]:
dataset_new.head()

In [ ]:
dataset_new.tail(20)

In [ ]:
dataset_new.tail(20).corr()

In [ ]:
dataset_new.tail()

In [ ]:
# split into train and test dataset
#dataset_train = dataset.iloc[0:270, 1:2]
dataset_train = df2.iloc[0:201, ]
#dataset_test = dataset.iloc[270:300,1:2]
dataset_test = df2.iloc[201:337,]

In [ ]:
dataset_train.drop('Close',axis=1,inplace=True)

In [ ]:
# train dataset
dataset_train.head()
dataset_train.shape
#dataset_train.tail()

In [ ]:
dataset_test.drop('Close',axis=1,inplace=True)

In [ ]:
# test dataset
dataset_test.head()
dataset_test.shape
#dataset_test.tail()

In [ ]:
# convert train dataset into an array
training_set = dataset_train.values
#training_set

In [ ]:
# perform feature scaling 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 201):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Masking

In [ ]:
# Initialising the RNN
regressor = Sequential()

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 4)))
regressor.add(Dropout(0.2))

In [ ]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [ ]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [ ]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [ ]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# Making predictions and visualising the results
# Getting the real FTSE prices for the period 28/03/2018 to 11/05/2018
real_ftse_price = dataset_test['Open'].values

In [ ]:
real_ftse_price

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Open','High','Low','Adj Close']], dataset_test[['Open','High','Low','Adj Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 196):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_ftse_price = regressor.predict(X_test)


In [ ]:
predicted_ftse_price.shape
#inputs.shape
df_zeros = np.zeros(shape=(len(predicted_ftse_price), 4) )
df_zeros[:,0] = predicted_ftse_price[:,0]
#df_zeros

In [ ]:
predicted_ftse_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
predicted_ftse_price

In [ ]:
# Visualising the results
plt.plot(real_ftse_price, color = 'red', label = 'Real FTSE Price')
plt.plot(predicted_ftse_price, color = 'blue', label = 'Predicted FTSE Price')
plt.title('FTSE Price Prediction')
plt.xlabel('Time')
plt.ylabel('FTSE Price')
plt.legend()
plt.show()

In [ ]:
# import the dataset
dataset = pd.read_csv("../input/ftse100/ftse.csv",index_col="Date")
dataset.head()
dataset.tail(40)

In [ ]:
ftse = dataset.dropna(axis=0)

In [ ]:
ftse.isnull().sum()

In [ ]:
ftse.head()

In [ ]:
ftse['Close'].mean()

In [ ]:
dataset.shape

In [ ]:
#dataset.fillna(-1,inplace=True)

In [ ]:
dataset.tail()

In [ ]:
dataset.drop(['Close','Volume'],axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.fillna(method='ffill',inplace=True)

In [ ]:
#dataset.iloc[300:321,]

In [ ]:
#dataset_new = dataset.merge(spy_open,on='Date',how='left',left_index=True,right_index=False, indicator=True)
#dataset_new.shape
#dataset_new.head()
#dataset_new.tail()

In [ ]:
#dataset_new.fillna(method='ffill',inplace=True)

In [ ]:
#dataset_new.tail()

In [ ]:
# split into train and test datasets
dataset_train = dataset.iloc[0:310, 1:5]
dataset_train.tail()
#dataset_train.shape

In [ ]:
dataset_test = dataset.iloc[310:321, 1:5]
dataset_test.tail()

In [ ]:
#dataset_train.drop('Close',axis=1,inplace=True)
#dataset_train.head()

In [ ]:
#dataset_train.fillna(method='bfill',inplace=True)

In [ ]:
#dataset_train.fillna(-1,inplace=True)

In [ ]:
#dataset_test.fillna(-1,inplace=True)

In [ ]:
dataset_train.head()

In [ ]:
dataset_train.tail()

In [ ]:
dataset_train.corr()

In [ ]:
#dataset_train.drop(['Volume','Open_spy'],axis=1,inplace=True)
#dataset_train.head()

In [ ]:
# convert into array
training_set = dataset_train.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
training_set_scaled

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 310):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
#X_train

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
#regressor.add(Masking(mask_value=0, input_shape = (X_train.shape[1], 4)))
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 4)))
#regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# test dataset
#dataset_test = dataset_new.iloc[270:320, 1:8]
#dataset_test.tail()
#dataset_test.shape
#dataset_test.head()

In [ ]:
#dataset_test.drop(['Close','Volume','Open_spy'],axis=1,inplace=True)
#dataset_test.head()

In [ ]:
real = dataset_test['Open'].values
#real

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Open','High','Low','Adj Close']], dataset_test[['Open','High','Low','Adj Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 71):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_ftse_price = regressor.predict(X_test)
#predicted_ftse_price = sc.inverse_transform(predicted_ftse_price)

In [ ]:
#predicted_ftse_price

In [ ]:
# invert transformation
df_zeros = np.zeros(shape=(len(predicted_ftse_price), 4) )
df_zeros[:,0] = predicted_ftse_price[:,0]
#df_zeros

In [ ]:
predicted_ftse_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
#predicted_ftse_price

In [ ]:
# Visualising the results
plt.plot(real, color = 'red', label = 'Imputed FTSE Price')
plt.plot(predicted_ftse_price, color = 'blue', label = 'Predicted FTSE Price')
plt.title('FTSE Price Prediction')
plt.xlabel('Time')
plt.ylabel('FTSE Price')
plt.legend()
plt.show()

In [ ]:
# import the GOLD dataset
dataset = pd.read_csv("../input/gold1/gold.csv",index_col='Date')
#dataset.head()
#dataset.tail(30)

In [ ]:
#dataset.tail(20)

In [ ]:
dataset_new = dataset.merge(ftse,on='Date',how='left',left_index=True,right_index=False, indicator=True)
#dataset_new.shape
dataset_new.head()
#dataset_new.tail()

In [ ]:
dataset_new.corr()

In [ ]:
dataset.shape

In [ ]:
dataset.Close.min()

In [ ]:
dataset.Close.mean()

In [ ]:
dataset.Close.max()

In [ ]:
dataset.Close.std()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
dataset['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=90.61, color='g', linestyle='-')
plt.axhline(y=76.72, color='r', linestyle='-')
plt.axhline(y=107.90, color='r', linestyle='-')

In [ ]:
dataset.drop('Close',axis=1,inplace=True)

In [ ]:
dataset.corr()

In [ ]:
#df = dataset.values

In [ ]:
# imputation using the mean values
from sklearn.preprocessing import Imputer, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
imputer = Imputer(strategy='mean')
df = imputer.fit_transform(dataset)
#model = make_pipeline(Imputer(strategy='mean'), PolynomialFeatures(2), LinearRegression())
#df = model.fit_transform(dataset)

In [ ]:
df1 = pd.DataFrame(df,columns=('Open','High','Low','Adj Close','Volume'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df1['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=90.61, color='g', linestyle='-')
plt.axhline(y=76.72, color='r', linestyle='-')
plt.axhline(y=107.90, color='r', linestyle='-')

In [ ]:
df2 = df1.drop('Volume',axis=1)

In [ ]:
df2.tail()

In [ ]:
#imputation of missing values
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
# using MICE
#df1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='mean',n_pmm_neighbors=5).complete(dataset) #probablistic moment matching
df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='mean',n_pmm_neighbors=5).complete(dataset) #fill in with samples from posterior predictive distribution

In [ ]:
df2 = pd.DataFrame(df1,columns=('Open','High','Low','Adj Close','Volume'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2['Adj Close'].plot(figsize=(16,6))
plt.axhline(y=90.61, color='g', linestyle='-')
plt.axhline(y=76.72, color='r', linestyle='-')
plt.axhline(y=107.90, color='r', linestyle='-')

In [ ]:
df2.drop('Volume',axis=1,inplace=True)

In [ ]:
#dataset.fillna(method='ffill',inplace=True)

In [ ]:
df2.shape

In [ ]:
dataset_train = df2.iloc[0:201,]
dataset_train.head()
#dataset_train.shape

In [ ]:
dataset_test = df2.iloc[201:270,]
dataset_test.tail()
#dataset_train.shape

In [ ]:
training_set = dataset_train.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 201):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 4)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# test values
test_values = dataset_test['Open'].values

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Open','High','Low','Adj Close']], dataset_test[['Open','High','Low','Adj Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 129):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_gold_price = regressor.predict(X_test)
#predicted_gold_price = sc.inverse_transform(predicted_gold_price)
#predicted_gold_price

In [ ]:
# invert transformation
df_zeros = np.zeros(shape=(len(predicted_gold_price), 4) )
df_zeros[:,0] = predicted_gold_price[:,0]

In [ ]:
predicted_gold_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
# Visualising the results
plt.plot(test_values, color = 'red', label = 'GOLD Price')
plt.plot(predicted_gold_price, color = 'blue', label = 'Predicted GOLD Price')
plt.axhline(y=90.61, color='g', linestyle='-')
plt.axhline(y=76.72, color='r', linestyle='-')
plt.axhline(y=107.90, color='r', linestyle='-')
plt.title('GOLD Price Prediction')
plt.xlabel('Time')
plt.ylabel('GOLD Price')
plt.legend()
plt.show()

In [ ]:
# Import DAX data
dax = pd.read_csv("../input/dax500/dax.csv",index_col='Date')
dax.head()

In [ ]:
dax.tail()

In [ ]:
dax.shape

In [ ]:
#non_missing_values = dax_new[:306].dropna(axis=0)
dax.dropna(axis=0,inplace=True)
dax.isnull().sum()

In [ ]:
#non_missing_values.tail()

In [ ]:
#missing_values = dax_new.iloc[306:337,:]

In [ ]:
#dataset2 = non_missing_values.append(missing_values)

In [ ]:
#dataset3 = dataset2.iloc[653:1300,:]

In [ ]:
#dataset2.isnull().sum()

In [ ]:
dax['Close'].min()

In [ ]:
dax['Close'].mean()

In [ ]:
dax['Close'].max()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
dax['Close'].plot(figsize=(16,6))
plt.axhline(y=12684, color='g', linestyle='-')
plt.axhline(y=11787, color='r', linestyle='-')
plt.axhline(y=13559, color='r', linestyle='-')

In [ ]:
#imputation of missing values
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
# using MICE
#df1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='random',n_pmm_neighbors=5).complete(dataset2) #probablistic moment matching
#df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='random',n_pmm_neighbors=5).complete(dataset2) #fill in with samples from posterior predictive distribution

In [ ]:
#df2 = pd.DataFrame(df1,columns=('Open_x','High_x','Low_x','Adj Close_x','Open_y'))

In [ ]:
#df2.tail()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#df2['Adj Close_x'].plot(figsize=(16,6))
#plt.axhline(y=12632, color='g', linestyle='-')
#plt.axhline(y=11787, color='r', linestyle='-')
#plt.axhline(y=13559, color='r', linestyle='-')

In [ ]:
#df2.drop(['Close','Volume'],axis=1,inplace=True)

In [ ]:
#df2.head()

In [ ]:
#df2.shape

In [ ]:
dataset_train = dax.iloc[0:201,]
dataset_train.tail()

In [ ]:
dataset_train.drop(['Adj Close','Volume'],axis=1,inplace=True)

In [ ]:
dataset_train.head()

In [ ]:
dataset_test = dax.iloc[201:257,]
dataset_test.tail()

In [ ]:
dataset_test.drop(['Adj Close','Volume'],axis=1,inplace=True)

In [ ]:
training_set = dataset_train.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 200):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 3])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 4)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose=1, validation_split=0.1)

In [ ]:
# test values
test_values = dataset_test['Close'].values

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Open','High','Low','Close']], dataset_test[['Open','High','Low','Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 112):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_dax_price = regressor.predict(X_test)

In [ ]:
# invert transformation
df_zeros = np.zeros(shape=(len(predicted_dax_price), 4) )
df_zeros[:,0] = predicted_dax_price[:,0]

In [ ]:
predicted_dax_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
# Visualising the results
plt.plot(test_values, color = 'red', label = 'DAX Price')
plt.plot(predicted_dax_price, color = 'blue', label = 'Predicted DAX Price')
plt.axhline(y=12684, color='g', linestyle='-')
plt.title('DAX Price Prediction')
plt.xlabel('Time')
plt.ylabel('DAX Price')
plt.legend()
plt.show()

In [ ]:
# add missing rows
test = dax
test.loc[-21] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

In [ ]:
test.drop(['Close','Adj Close'],axis=1,inplace=True)

In [ ]:
test.describe()

In [ ]:
#imputation of missing values
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
# using MICE
test1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='random',n_pmm_neighbors=5).complete(test) #probablistic moment matching
#df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='random',n_pmm_neighbors=5).complete(dataset2) #fill in with samples from posterior predictive distribution

In [ ]:
test.describe()

In [ ]:
test2 = pd.DataFrame(test1,columns=('Open','High','Low','Volume'))

In [ ]:
test2.drop('Volume',axis=1,inplace=True)
test2.describe()

In [ ]:
test3 = test2.tail(21)

In [ ]:
test3.shape

In [ ]:
dataset_total_new = pd.concat((dataset_total[['Open','High','Low','Close']], test3[['Open','High','Low']]), axis = 0)

In [ ]:
dataset_total_new

In [ ]:

inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-4,4)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 112):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))
predicted_dax_price = regressor.predict(X_test)

In [ ]:
# Import gold data
gold = pd.read_csv("../input/futures/gold_futures5.csv",index_col='Date')
#dataset.head()

In [ ]:
gold.tail()

In [ ]:
# reverse the dataset
gold_r = gold.iloc[::-1]

In [ ]:
gold_r.head()

In [ ]:
gold_r.shape

In [ ]:
gold_r.Value.min()

In [ ]:
gold_r.Value.mean()

In [ ]:
gold_r.Value.max()

In [ ]:
gold_r.Value.std()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
gold_r['Value'].plot(figsize=(16,6))
plt.axhline(y=1296, color='g', linestyle='-')
plt.axhline(y=1211, color='r', linestyle='-')
plt.axhline(y=1355, color='r', linestyle='-')

In [ ]:
#imputation of missing values
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
# using MICE
#df1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='median',n_pmm_neighbors=5).complete(gold_r) #probablistic moment matching
df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='median',n_pmm_neighbors=5).complete(gold_r) #fill in with samples from posterior predictive distribution

In [ ]:
df2 = pd.DataFrame(df1)
df2.columns = ['Value']

In [ ]:
df2.tail()

In [ ]:
df2.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2['Value'].plot(figsize=(16,6))
plt.axhline(y=1295, color='g', linestyle='-')
plt.axhline(y=1211, color='r', linestyle='-')
plt.axhline(y=1355, color='r', linestyle='-')

In [ ]:
df2.shape

In [ ]:
dataset_train = df2.iloc[0:201,]
dataset_train.tail()

In [ ]:
dataset_test = df2.iloc[201:278,]
dataset_test.tail()

In [ ]:
training_set = dataset_train.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 201):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# test values
test_values = dataset_test['Value'].values

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train['Value'], dataset_test['Value']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 137):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_dax_price = regressor.predict(X_test)

In [ ]:
# invert transformation
df_zeros = np.zeros(shape=(len(predicted_dax_price), 1) )
df_zeros[:,0] = predicted_dax_price[:,0]

In [ ]:
predicted_dax_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
# Visualising the results
plt.plot(test_values, color = 'red', label = 'Gold futures Price')
plt.plot(predicted_dax_price, color = 'blue', label = 'Predicted Gold futures Price')
plt.axhline(y=1299, color='g', linestyle='-')
plt.axhline(y=1270, color='r', linestyle='-')
plt.axhline(y=1355, color='r', linestyle='-')
plt.title('Gold futures Price Prediction')
plt.xlabel('Time')
plt.ylabel('Gold futures Price')
plt.legend()
plt.show()

In [ ]:
dataset_new = gold_r.merge(dataset,on='Date',how='left',left_index=True,right_index=False, indicator=True)

In [ ]:
dataset_new.corr()

In [ ]:
dataset_new.head()

In [ ]:
dataset_new.tail(20)

In [ ]:
dataset_new.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
dataset_new['Value'].plot(figsize=(16,6))
plt.axhline(y=1300, color='g', linestyle='-')
plt.axhline(y=1270, color='r', linestyle='-')
plt.axhline(y=1320, color='r', linestyle='-')

In [ ]:
dataset_new.shape

In [ ]:
dataset_new.drop(['Close','Volume','_merge'],axis=1,inplace=True)

In [ ]:
dataset_new.tail()

In [ ]:
#imputation of missing values
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
# using MICE
df1 = MICE(n_imputations=100,impute_type='pmm',init_fill_method='median',n_pmm_neighbors=5).complete(dataset_new) #probablistic moment matching
#df1 = MICE(n_imputations=100,impute_type='col',init_fill_method='median',n_pmm_neighbors=5).complete(dataset_new) #fill in with samples from posterior predictive distribution

In [ ]:
df2 = pd.DataFrame(df1,columns=('Value','Open','High','Low','Adj Close'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df2['Value'].plot(figsize=(16,6))
plt.axhline(y=1300, color='g', linestyle='-')
plt.axhline(y=1270, color='r', linestyle='-')
plt.axhline(y=1320, color='r', linestyle='-')

In [ ]:
dataset_train = df2.iloc[0:201,]

In [ ]:
dataset_test = df2.iloc[201:278,]

In [ ]:
training_set = dataset_train.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 201):
    X_train.append(training_set_scaled[i-60:i, :])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))

In [ ]:
# Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Build and fit the RNN
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# test values
test_values = dataset_test['Value'].values

In [ ]:
# Getting the predicted prices
dataset_total = pd.concat((dataset_train[['Value','Open','High','Low','Adj Close']], dataset_test[['Value','Open','High','Low','Adj Close']]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-5,5)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 137):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
predicted_dax_price = regressor.predict(X_test)

In [ ]:
# invert transformation
df_zeros = np.zeros(shape=(len(predicted_dax_price), 5) )
df_zeros[:,0] = predicted_dax_price[:,0]

In [ ]:
predicted_dax_price = sc.inverse_transform(df_zeros)[:,0]

In [ ]:
# Visualising the results
plt.plot(test_values, color = 'red', label = 'Gold futures Price')
plt.plot(predicted_dax_price, color = 'blue', label = 'Predicted Gold futures Price')
plt.axhline(y=1300, color='g', linestyle='-')
plt.axhline(y=1320, color='r', linestyle='-')
plt.axhline(y=1270, color='r', linestyle='-')
plt.title('Gold futures Price Prediction')
plt.xlabel('Time')
plt.ylabel('Gold futures Price')
plt.legend()
plt.show()